In [1]:
#import urllib2
from io import StringIO
import os
import xmltodict
import pandas as pd
import geopandas as gpd
import windrose

from datetime import datetime 
import statsmodels.api as sm

from scipy.stats import linregress
import matplotlib.pyplot as plt
import numpy as np
import sys
import requests
import glob

from pyproj import CRS
from pyproj import Transformer
from shapely.geometry import Point

import rasterio

from rasterio.plot import show
from rasterio.mask import mask
from rasterio.plot import show_hist
from shapely.geometry import box
from windrose import WindroseAxes
#import pycrs
#import earthpy as et
#import earthpy.plot as ep

from scipy.optimize import curve_fit

from sklearn import mixture
import itertools
from scipy import linalg
import matplotlib as mpl

import pathlib

import plotly.express as px


#import pymannkendall as mk
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 15, 10

In [5]:
from dbfread import DBF


In [ ]:
  import dbf

ruta = 'Y:/SAITBC/'
table = dbf.Table(ruta+'clientes.dbf')
tablita = table.open()

In [9]:
str(file.name).split(".")[0].split("_")

['fields', 'actual', 'et', '2016']

In [24]:
gis_path = pathlib.Path("G:/Shared drives/UGS_Groundwater/Projects/Eddy_Covariance/GIS/Cottonwood/SWB_Zonal/")

def summarize_zones(keyword, gis_path):
    dfs = {}
    
    for file in gis_path.glob(f'{keyword}*.dbf'):
        name_meta = str(file.name).split(".")[0].split("_")
        sum_type = name_meta[0]
        year = int(name_meta[-1])
        middle = " ".join(name_meta[1:-1])
    
        dbf = DBF(file, load=True)
        dbf = pd.DataFrame(dbf)
        dbf['year'] = year
        dbf['component'] = middle
        dbf['summary type'] = sum_type
        dfs[f"{sum_type}-{middle}-{year}"] = dbf
    
    return pd.concat(dfs)

In [28]:
field_summary = summarize_zones('fields',gis_path)
huc_summary = summarize_zones('hucs',gis_path)



In [29]:
huc_summary.huc_12.unique()

array(['140600070603', '140600070601', '140600070602', '140600070704',
       '140600070705', '140600090104', '140600070701', '140600070702',
       '140600090105', '140600070706', '140600090204', '140600090107',
       '140600090206', '140600090205', '140600090106', '140600090209',
       '140600090108', '140600090208', '140600090207', '140600090305',
       '140600090304', '140600090102', '140600090101', '140600090103',
       '140600090202', '140600090201', '140600090203', '140600090301',
       '140600090303', '140600090302', '140600090210'], dtype=object)

In [42]:
huc_summary['measure_m'] = huc_summary['MEAN']*0.0254
huc_summary['vol_m3'] = huc_summary['MEAN']*0.0254*huc_summary['AREA']
huc_summary['vol_acft'] = huc_summary['vol_m3']*0.000810714


In [43]:
huc_summary[huc_summary['huc_12']=='140600090210']

,,huc_12,ZONE_CODE,COUNT,AREA,MIN,MAX,RANGE,MEAN,STD,SUM,MEDIAN,PCT95,PCT5,year,component,summary type,measure_m,vol_m3,vol_acft
hucs-actual et-2016,30,140600090210,31,6606.0,66060000.0,3.795709,38.359695,34.563986,17.385005,4.753877,114845.340547,15.423885,22.775218,12.363363,2016,actual et,hucs,0.441579,2.917072e+07,23649.108256
hucs-actual et-2017,30,140600090210,31,6606.0,66060000.0,4.689202,42.797298,38.108097,22.277983,5.038390,147168.355795,19.627496,29.157314,17.444100,2017,actual et,hucs,0.565861,3.738076e+07,30305.107386
hucs-actual et-2018,30,140600090210,31,6606.0,66060000.0,2.652622,44.800865,42.148243,14.757069,5.279264,97485.197487,13.721428,19.436362,10.478353,2018,actual et,hucs,0.374830,2.476124e+07,20074.284056
hucs-gross precipitation-2020,30,140600090210,31,6606.0,66060000.0,13.298818,19.925987,6.627170,15.410642,1.676528,101802.702951,14.990155,18.965357,13.518899,2020,gross precipitation,hucs,0.391430,2.585789e+07,20963.350636
hucs-gross precipitation-2021,30,140600090210,31,6606.0,66060000.0,14.012990,20.677956,6.664966,16.078169,1.622297,106212.383688,15.420077,19.386219,14.308267,2021,gross precipitation,hucs,0.408385,2.697795e+07,21871.398073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
hucs-runon-2017,30,140600090210,31,6606.0,66060000.0,0.000000,59.073540,59.073540,1.881773,3.539007,12430.992099,0.102527,8.202464,0.000000,2017,runon,hucs,0.047797,3.157472e+06,2559.806750
hucs-rainfall-2020,30,140600090210,31,6606.0,66060000.0,1.166929,3.927953,2.761024,2.656233,0.829109,17547.074551,3.244488,3.444882,1.314173,2020,rainfall,hucs,0.067468,4.456957e+06,3613.317385
hucs-reference ET0-2016,30,140600090210,31,6606.0,66060000.0,26.627886,38.359695,11.731810,33.151745,3.118541,219000.427610,33.545635,37.832668,27.555138,2016,reference ET0,hucs,0.842054,5.562611e+07,45096.865018
hucs-rainfall-2022,30,140600090210,31,6606.0,66060000.0,6.372835,8.737403,2.364568,7.870204,0.543832,51990.566138,8.012992,8.570078,6.524016,2022,rainfall,hucs,0.199903,1.320560e+07,10705.967878


In [30]:
feeds_joes_valley = ['140600090202', '140600090201','140600090210']

In [32]:
print(huc_summary['component'].unique())

['actual et' 'gross precipitation' 'interception' 'irrigation'
 'net infiltration' 'reference ET0' 'rainfall' 'rejected net infiltration'
 'runon' 'runoff outside' 'runoff' 'snowfall']


In [44]:
upper_joes = huc_summary[huc_summary['huc_12'].isin(feeds_joes_valley)]

In [48]:
upper_joes.groupby(['component','year']).sum()

huc_12  ZONE_CODE    COUNT  \
component year                                                             
actual et 2016  140600090202140600090201140600090210         82  28425.0   
          2017  140600090202140600090201140600090210         82  28425.0   
          2018  140600090202140600090201140600090210         82  28425.0   
          2019  140600090202140600090201140600090210         82  28425.0   
          2020  140600090202140600090201140600090210         82  28425.0   
...                                              ...        ...      ...   
snowfall  2018  140600090202140600090201140600090210         82  28425.0   
          2019  140600090202140600090201140600090210         82  28425.0   
          2020  140600090202140600090201140600090210         82  28425.0   
          2021  140600090202140600090201140600090210         82  28425.0   
          2022  140600090202140600090201140600090210         82  28425.0   

                       AREA        MIN         MAX      RANGE       MEAN  \
component year                                                             
actual et 2016  284250000.0  22.194455  103.274780  81.080325  58.832570   
          2017  284250000.0  28.477053  117.445923  88.968870  72.246502   
          2018  284250000.0  17.347257  111.110445  93.763188  48.956847   
          2019  284250000.0  26.279519  112.157005  85.877486  70.079593   
          2020  284250000.0  16.946743  113.066227  96.119484  50.705374   
...                     ...        ...         ...        ...        ...   
snowfall  2018  284250000.0  12.505117   48.284646  35.779529  30.063295   
          2019  284250000.0  40.985432  111.725986  70.740555  75.871159   
          2020  284250000.0  31.098426   63.206297  32.107871  46.632823   
          2021  284250000.0  25.798030   46.583858  20.785828  34.864158   
          2022  284250000.0  30.439371   63.539368  33.099997  46.156454   

                      STD            SUM     MEDIAN       PCT95       PCT5  \
component year                                                               
actual et 2016  12.752989  568814.891129  58.946840   73.803839  41.258583   
          2017  13.860469  695196.001451  70.588269   89.920969  53.959045   
          2018  12.445347  487349.712239  46.897328   66.057403  33.999700   
          2019  13.413140  669769.191601  69.259130   88.461761  51.253712   
          2020  13.239577  499984.671021  50.065020   68.023306  35.963338   
...                   ...            ...        ...         ...        ...   
snowfall  2018   8.686385  308419.744452  29.561811   44.214175  15.360237   
          2019  18.218299  767886.697146  75.080318  105.643311  46.627169   
          2020   7.945978  464925.273002  46.572046   59.638590  33.474802   
          2021   4.897126  344707.416301  35.019293   42.933464  27.332678   
          2022   8.673116  463496.396171  46.406301   59.770081  33.351970   

                summary type  measure_m        vol_m3       vol_acft  
component year                                                        
actual et 2016  hucshucshucs   1.494347  1.444790e+08  117131.133695  
          2017  hucshucshucs   1.835061  1.765798e+08  143155.703305  
          2018  hucshucshucs   1.243504  1.237868e+08  100355.713591  
          2019  hucshucshucs   1.780022  1.701214e+08  137919.780141  
          2020  hucshucshucs   1.287916  1.269961e+08  102957.521436  
...                      ...        ...           ...            ...  
snowfall  2018  hucshucshucs   0.763608  7.833862e+07   63510.211995  
          2019  hucshucshucs   1.927127  1.950432e+08  158124.269931  
          2020  hucshucshucs   1.184474  1.180910e+08   95738.042655  
          2021  hucshucshucs   0.885550  8.755568e+07   70982.618588  
          2022  hucshucshucs   1.172374  1.177281e+08   95443.806401  

[84 rows x 17 columns]